In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import os


url = "https://dev.to/latest"
ua = UserAgent()
userAgent = ua.random
headers = {"user-agent": userAgent}
page = requests.get(url, headers = headers)

soup = BeautifulSoup(page.content, "html.parser")

In [20]:
blog_box = soup.find_all("div", class_ = "crayons-story__body")

links = []
titles = []
time_uploaded = []
authors = []
tags = []
reading_times = []

for box in blog_box:
    #links
    if box.find("h2", class_ = "crayons-story__title") is not None:
        link = box.find("h2", class_ = "crayons-story__title").a
        link = link["href"]
        links.append(link)
    else:
        links.append("None")
        
    #titles
    if box.find("h2", class_ = "crayons-story__title") is not None:
        title = box.find("h2", class_ = "crayons-story__title")
        titles.append(title.text.replace("\n", "").strip())
    else:
        titles.append("None")
        
    #time_uploaded
    if box.find("time", attrs = {"datetime": True}) is not None:
        time_upload = box.find("time", attrs = {"datetime": True})
        time_upload = time_upload["datetime"]
        time_uploaded.append(time_upload)
    else:
        time_uploaded.append("None")
        
    #authors
    if box.find("a", class_ = "crayons-story__secondary fw-medium m:hidden") is not None:
        author = box.find("a", class_ = "crayons-story__secondary fw-medium m:hidden")
        authors.append(author.text.replace("\n", "").strip())
    else:
        authors.append("None")
        
    #tags
    if box.find("div", class_ = "crayons-story__tags") is not None:
        tag = box.find("div", class_ = "crayons-story__tags")
        tags.append(tag.text.replace("\n", " ").strip())
    else:
        tags.append("None")
        
    #reading_times
    if box.find("div",class_ = "crayons-story__save") is not None:
        reading_time = box.find("div",class_ = "crayons-story__save")
        reading_times.append(reading_time.text.replace("\n", "").strip())
    else:
        reading_times.append("None")

In [22]:
blog_df = pd.DataFrame(
    {
        "Link": links,
        "Title": titles,
        "Time_Uploaded": time_uploaded,
        "Author": authors,
        "Tag": tags,
        "Reading_Time": reading_times
    }
)

blog_df = blog_df[blog_df["Link"] != "None"]
blog_df

,Link,Title,Time_Uploaded,Author,Tag,Reading_Time
1,https://dev.to/indraphoton/how-my-ai-agent-tra...,🤖How My AI Agent Transformed Video Research fr...,2025-06-11T09:29:45Z,INDRANIL MAITI,#ai #aiagent #python #beginners,3 min read
2,https://dev.to/rac/big-data-project-ideas-for-...,Big Data Project Ideas for Students and Beginners,2025-06-11T09:26:37Z,Zack Rac,#programming #beginners #javascript #python,3 min read
3,https://dev.to/gauravchadhry/crossbuild-ui-fig...,🚀 Crossbuild UI – Figma-Inspired Expo + React ...,2025-06-11T09:24:34Z,Gaurav Singh,#ui #reactnative #mobile #webdev,1 min read
4,https://dev.to/rac/best-online-courses-and-cer...,Best Online Courses and Certifications for Lea...,2025-06-11T09:22:46Z,Zack Rac,#programming #javascript #ai #beginners,3 min read
5,https://dev.to/gifhow/top-5-principles-for-eff...,Top 5 Principles for Effective AI-Powered “Vib...,2025-06-11T09:18:51Z,GifHow,#webdev #programming #ai #tooling,5 min read
6,https://dev.to/yukinisihikawa/github-repo-is-l...,GitHub repo is live! If this feels useful or i...,2025-06-11T09:17:49Z,Yuki Nishikawa,#showdev #webdev #opensource #discuss,1 min read
7,https://dev.to/excel24x7/how-to-use-a-formula-...,How to Use a Formula to Convert Text to Senten...,2025-06-11T09:15:00Z,Vigneshwaran Vijayakumar,#tips #beginners #excel #office,3 min read
8,https://dev.to/violetbrown/-1li5,Vitara AI - A Vibe Coding Tools,2025-06-11T09:14:40Z,Violet Brown,#vibecoding #vibecodingtools #codingtool #prog...,1 min read
9,https://dev.to/alvarosabu/secret-2j07,Secret,2025-06-11T09:12:23Z,Alvaro Saburido,,1 min read
10,https://dev.to/mufeng/building-a-scalable-hype...,Building a Scalable Hypercasual Game Portal fr...,2025-06-11T09:12:15Z,mufeng,#webdev #h5 #gamedev #h5game,4 min read
